# california-coronavirus-data examples

By [Ben Welsh](https://palewi.re/who-is-ben-welsh)

A demonstration of how to use Python to work with the Los Angeles Times' independent tally of coronavirus cases in California published on GitHub at [datadesk/california-coronavirus-data](https://github.com/datadesk/california-coronavirus-data#state-cdph-totalscsv). To run this notebook immediately in the cloud,  click the [Binder](https://mybinder.org/) launcher below.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/datadesk/california-coronavirus-data/master?urlpath=lab/tree/notebooks/examples.ipynb)

## Import Python tools

Our data analysis and plotting tools

In [1]:
import pandas as pd
import altair as alt

Customizations to the Altair theme

In [2]:
import altair_latimes as lat

In [3]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

## Import data

Read in the agency totals

In [4]:
agency_df = pd.read_csv(
    "latimes-agency-totals.csv",
    parse_dates=["date"]
)

In [5]:
agency_df.head()

,agency,county,fips,date,confirmed_cases,deaths,did_not_update
0,Alameda,Alameda,1,2020-06-09,3878,101.0,NaN
1,Berkeley,Alameda,1,2020-06-09,107,1.0,NaN
2,Alpine,Alpine,3,2020-06-09,1,0.0,True
3,Amador,Amador,5,2020-06-09,10,0.0,NaN
4,Butte,Butte,7,2020-06-09,71,1.0,NaN


In [6]:
agency_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5549 entries, 0 to 5548
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   agency           5549 non-null   object        
 1   county           5549 non-null   object        
 2   fips             5549 non-null   int64         
 3   date             5549 non-null   datetime64[ns]
 4   confirmed_cases  5549 non-null   int64         
 5   deaths           5548 non-null   float64       
 6   did_not_update   747 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 303.6+ KB


## Aggregate data

### By state

Lump all the agencies together and you get the statewide totals.

In [7]:
state_df = agency_df.groupby(["date"]).agg({
    "confirmed_cases": "sum",
    "deaths": "sum"
}).reset_index()

In [8]:
state_df.head()

,date,confirmed_cases,deaths
0,2020-01-26,2,0.0
1,2020-01-27,3,0.0
2,2020-01-28,3,0.0
3,2020-01-29,4,0.0
4,2020-01-30,4,0.0


In [9]:
state_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             136 non-null    datetime64[ns]
 1   confirmed_cases  136 non-null    int64         
 2   deaths           136 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 3.3 KB


### By county

Three cities &mdash; Berkeley, Long Beach and Pasadena &mdash; run independent public health departments. Calculating county-level totals requires grouping them with their local peers.

In [10]:
county_df = agency_df.groupby(["date", "county"]).agg({
    "confirmed_cases": "sum",
    "deaths": "sum"
}).reset_index()

In [11]:
county_df.head()

,date,county,confirmed_cases,deaths
0,2020-01-26,Alameda,0,0.0
1,2020-01-26,Calaveras,0,0.0
2,2020-01-26,Contra Costa,0,0.0
3,2020-01-26,Humboldt,0,0.0
4,2020-01-26,Los Angeles,1,0.0


In [12]:
county_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5240 entries, 0 to 5239
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             5240 non-null   datetime64[ns]
 1   county           5240 non-null   object        
 2   confirmed_cases  5240 non-null   int64         
 3   deaths           5240 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 163.9+ KB


## Chart the statewide totals over time

In [13]:
# Create a base chart with the common x-axis
chart = alt.Chart(state_df).encode(
    x=alt.X("date:T", title=None)
)

# Create the cases line
cases = chart.mark_line(color=lat.palette['default']).encode(
    y=alt.Y("confirmed_cases:Q", title="Confirmed cases")
)

# Create the deaths line
deaths = chart.mark_line(color=lat.palette['schemes']['ice-7'][3]).encode(
    y=alt.Y("deaths:Q", title="Deaths")
)

# Combine them into a single chart
(cases & deaths).properties(title="Statewide cumulative totals")

alt.VConcatChart(...)

## Chart the county totals

First on a linear scale

In [14]:
# Create the base chart
alt.data_transformers.disable_max_rows()
chart = alt.Chart(county_df).mark_line().encode(
    x=alt.X("date:T", title=None),
    color=alt.Color("county:N", title="County", legend=None)
)

# The cases line
cases = chart.encode(
    y=alt.Y(
        "confirmed_cases:Q",
        title="Confirmed cases"
    ),
)

# The deaths line
deaths = chart.mark_line().encode(
    y=alt.Y("deaths:Q", title="Deaths"),
)

# Combined into a chart
(cases & deaths).properties(title="Cumulative totals by county")

alt.VConcatChart(...)

Again on a logarithmic scale

In [15]:
# Make a base chart
chart = alt.Chart(county_df).mark_line().encode(
    x=alt.X("date:T", title=None),
    color=alt.Color("county:N", title="County", legend=None)
)

# The cases lines
cases = chart.transform_filter(alt.datum.confirmed_cases > 0).encode(
    y=alt.Y(
        "confirmed_cases:Q",
        scale=alt.Scale(type='log'),
        title="Confirmed cases"
    ),
)

# The deaths lines
deaths = chart.transform_filter(alt.datum.deaths > 0).encode(
    y=alt.Y(
        "deaths:Q",
        scale=alt.Scale(type='log'),
        title="Deaths"
    ),
)

# Slapping them together
(cases & deaths).properties(title="Cumulative totals by county")

alt.VConcatChart(...)

A common technique for clarifying these charts to begin each line on the day the county hit a minimum number. Let's try it with 10.

In [16]:
day_10_df = county_df[
    # Filter down to only days with 10 or more cumulative cases
    county_df.confirmed_cases >= 10
].groupby(
    # And then get the minimum date for each county
    'county'
).date.min().reset_index()

Merge that date to each row in the data.

In [17]:
county_date_diff_df = county_df.merge(
    day_10_df,
    how='inner',
    on='county',
    suffixes=['', '_gte_10_cases']
)

Calculate each day's distance from its tenth day.

In [18]:
county_date_diff_df['days_since_10'] = (
    county_date_diff_df.date - county_date_diff_df.date_gte_10_cases
).dt.days

Chart it.

In [19]:
alt.Chart(county_date_diff_df).transform_filter(
    # Only keep everything once they hit 10 cases
    alt.datum.days_since_10 >= 0
).mark_line().encode(
    x=alt.X(
        "days_since_10:O",
        title="Days since 10th case"
    ),
    y=alt.Y(
        "confirmed_cases:Q",
        scale=alt.Scale(type='log'),
        title="Confirmed cases"
    ),
    color=alt.Color("county:N", title="County", legend=None)
).properties(title="Cumulative totals by county")

alt.Chart(...)

## County trends on a linear 'Pez' plot

Fill in any date gaps so that every county has a row for every date.

In [20]:
backfilled_county_df = county_df.set_index([
    "county",
    "date"
]).unstack("county").fillna(0).stack("county").reset_index()

Calculate the rolling change in each county.

In [21]:
chronological_county_df = backfilled_county_df.sort_values(["county", "date"])

Calculate the daily change in each county.

In [22]:
chronological_county_df['new_confirmed_cases'] = chronological_county_df.groupby("county").confirmed_cases.diff()

Let's chill that out as a seven-day average.

In [23]:
chronological_county_df['new_confirmed_cases_rolling_average'] = chronological_county_df.new_confirmed_cases.rolling(7).mean()

In [24]:
chronological_county_df.dtypes

date                                   datetime64[ns]
county                                         object
confirmed_cases                               float64
deaths                                        float64
new_confirmed_cases                           float64
new_confirmed_cases_rolling_average           float64
dtype: object

Make the chart.

In [25]:
alt.Chart(chronological_county_df, title="New cases by day").mark_rect(stroke=None).encode(
    x=alt.X(
        'date:O',
        axis=alt.Axis(
            ticks=False,
            grid=False,
            labels=False,
        ),
        title=None
    ),
    y=alt.Y(
        'county:N',
        title="County",
        axis=alt.Axis(ticks=False, grid=False, labelPadding=5)
    ),
    color=alt.Color(
        "new_confirmed_cases_rolling_average:Q",
        scale=alt.Scale(
            type="threshold",
            domain=[0, 3, 10, 25, 50, 100, 500],
            scheme="blues"
        ),
        title="New cases (7-day average)"
    )
).properties(height=800)

alt.Chart(...)

## Chart new cases and deaths

Calculate the number of new cases each day using panda's [diff](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.diff.html) method.

In [26]:
state_df['new_confirmed_cases'] = state_df.confirmed_cases.diff()

Do the same for deaths

In [27]:
state_df['new_deaths'] = state_df.deaths.diff()

Now calculate the moving seven-day average of each using panda's [rolling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) method.

In [28]:
state_df['new_confirmed_cases_rolling_average'] = state_df.new_confirmed_cases.rolling(7).mean()

In [29]:
state_df['new_deaths_rolling_average'] = state_df.new_deaths.rolling(7).mean()
state_df
state_df.dtypes

date                                   datetime64[ns]
confirmed_cases                                 int64
deaths                                        float64
new_confirmed_cases                           float64
new_deaths                                    float64
new_confirmed_cases_rolling_average           float64
new_deaths_rolling_average                    float64
dtype: object

In [30]:
imperial = chronological_county_df[chronological_county_df['county'] == 'Imperial']
imperial['new_deaths'] = imperial.deaths.diff()
imperial['new_deaths_rolling_average'] = imperial.new_deaths.rolling(7).mean()
imperial

C:\Users\Doug Smith\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Doug Smith\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,date,county,confirmed_cases,deaths,new_confirmed_cases,new_confirmed_cases_rolling_average,new_deaths,new_deaths_rolling_average
12,2020-01-26,Imperial,0.0,0.0,NaN,NaN,NaN,NaN
70,2020-01-27,Imperial,0.0,0.0,0.0,NaN,0.0,NaN
128,2020-01-28,Imperial,0.0,0.0,0.0,NaN,0.0,NaN
186,2020-01-29,Imperial,0.0,0.0,0.0,NaN,0.0,NaN
244,2020-01-30,Imperial,0.0,0.0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...
7610,2020-06-05,Imperial,2568.0,33.0,75.0,133.428571,3.0,0.857143
7668,2020-06-06,Imperial,2568.0,33.0,0.0,133.428571,0.0,0.857143
7726,2020-06-07,Imperial,2568.0,33.0,0.0,133.428571,0.0,0.857143
7784,2020-06-08,Imperial,3013.0,38.0,445.0,180.285714,5.0,1.285714


Put it all together on the chart 

In [31]:
# One base chart object with the data they all share
chart = alt.Chart(state_df).encode(
    x=alt.X("date:T", title=None),
)

# The new cases bars
cases_bars = chart.mark_bar(color=lat.palette['default']).encode(
    y=alt.Y(
        "new_confirmed_cases:Q",
        title="New confirmed cases"
    )
)

# The cases rolling average
cases_line = chart.mark_line(color=lat.palette['accent']).encode(
    y=alt.Y(
        "new_confirmed_cases_rolling_average:Q",
        title="7-day average"
    )
)

# The new deaths bars
deaths_bars = chart.mark_bar(color=lat.palette['schemes']['ice-7'][3]).encode(
    y=alt.Y(
        "new_deaths:Q",
        title="New deaths"
    )
)

# The deaths rolling average
deaths_line = chart.mark_line(color=lat.palette['schemes']['ice-7'][6]).encode(
    y=alt.Y(
        "new_deaths_rolling_average:Q",
        title="7-day average"
    )
)

# Combine it all together into one paired chart
((cases_bars + cases_line) & (deaths_bars + deaths_line)).properties(
    title="New case and deaths statewide by day"
)

alt.VConcatChart(...)

In [32]:
# One base chart object with the data they all share
chart = alt.Chart(imperial).encode(
    x=alt.X("date:T", title=None),
)

# The new cases bars
cases_bars = chart.mark_bar(color=lat.palette['default']).encode(
    y=alt.Y(
        "new_confirmed_cases:Q",
        title="New confirmed cases"
    )
)

# The cases rolling average
cases_line = chart.mark_line(color=lat.palette['accent']).encode(
    y=alt.Y(
        "new_confirmed_cases_rolling_average:Q",
        title="7-day average"
    )
)

# The new deaths bars
deaths_bars = chart.mark_bar(color=lat.palette['schemes']['ice-7'][3]).encode(
    y=alt.Y(
        "new_deaths:Q",
        title="New deaths"
    )
)

# The deaths rolling average
deaths_line = chart.mark_line(color=lat.palette['schemes']['ice-7'][6]).encode(
    y=alt.Y(
        "new_deaths_rolling_average:Q",
        title="7-day average"
    )
)

# Combine it all together into one paired chart
((cases_bars + cases_line) & (deaths_bars + deaths_line)).properties(
    title="New case and deaths Imperial County by day"
)

alt.VConcatChart(...)

In [33]:
care_df = pd.read_csv(
    "cdph-care.csv")
care_df

,date,name,county,TYPE,fips,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths
0,4/19/2020,EAST BAY POST-ACUTE,Alameda,SN,1,23,25,0,0
1,4/19/2020,EXCELL HEALTH CARE CENTER,Alameda,SN,1,1,1,0,0
2,4/19/2020,GARFIELD NEUROBEHAVIORAL CENTER,Alameda,SN,1,1,0,0,0
3,4/19/2020,GATEWAY CARE & REHABILITATION CENTER,Alameda,SN,1,33,69,0,0
4,4/19/2020,MERCY RETIREMENT & CARE CENTER,Alameda,SN,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...
45047,6/11/2020,RIVER BEND NURSING CENTER,Yolo,SN,113,0,0,0,0
45048,6/11/2020,STOLLWOOD CONVALESCENT HOSPITAL,Yolo,SN,113,13,32,1,15
45049,6/11/2020,UNIVERSITY RETIREMENT COMMUNITY AT DAVIS,Yolo,SN,113,0,0,0,0
45050,6/11/2020,WOODLAND SKILLED NURSING FACILITY,Yolo,SN,113,0,0,0,0


In [34]:
care_df.dtypes

date                        object
name                        object
county                      object
TYPE                        object
fips                         int64
staff_confirmed_cases        int64
patients_confirmed_cases     int64
staff_deaths                 int64
patients_deaths              int64
dtype: object

In [43]:
care_df['new_staff_cases_average'] = care_df.staff_confirmed_cases.rolling(7).mean()
care_df['new_patients_cases_average'] = care_df.patients_confirmed_cases.rolling(7).mean()
care_df['new_staff_deaths_average'] = care_df.staff_deaths.rolling(7).mean()
care_df['new_patients_deaths_average'] = care_df.patients_deaths.rolling(7).mean()
county_care_df = care_df[['date', 'county', 'new_staff_cases_average', 'new_patients_cases_average', 'new_staff_deaths_average', 'new_patients_deaths_average']]
county_care_df.fillna(0)

,date,county,new_staff_cases_average,new_patients_cases_average,new_staff_deaths_average,new_patients_deaths_average
0,4/19/2020,Alameda,0.000000,0.000000,0.000000,0.000000
1,4/19/2020,Alameda,0.000000,0.000000,0.000000,0.000000
2,4/19/2020,Alameda,0.000000,0.000000,0.000000,0.000000
3,4/19/2020,Alameda,0.000000,0.000000,0.000000,0.000000
4,4/19/2020,Alameda,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
45047,6/11/2020,Yolo,0.571429,0.142857,0.000000,0.000000
45048,6/11/2020,Yolo,2.285714,4.714286,0.142857,2.142857
45049,6/11/2020,Yolo,2.142857,4.571429,0.142857,2.142857
45050,6/11/2020,Yolo,2.000000,4.571429,0.142857,2.142857


In [44]:
county_care_df.to_csv('county_care.csv')